### Random Acts of Pizza - Baseline Model & EDA
## Authors: Ben Arnoldy, Mary Boardman, Zach Merritt, and Kevin Gifford
#### Kaggle Competition Description:
In machine learning, it is often said there are no free lunches. How wrong we were.

This competition contains a dataset with 5671 textual requests for pizza from the Reddit community Random Acts of Pizza together with their outcome (successful/unsuccessful) and meta-data. Participants must create an algorithm capable of predicting which requests will garner a cheesy (but sincere!) act of kindness.

"I'll write a poem, sing a song, do a dance, play an instrument, whatever! I just want a pizza," says one hopeful poster. What about making an algorithm?

Kaggle is hosting this competition for the machine learning community to use for fun and practice. This data was collected and graciously shared by Althoff et al. (Buy them a pizza -- data collection is a thankless and tedious job!) We encourage participants to explore their accompanying paper and ask that you cite the following reference in any publications that result from your work:

Tim Althoff, Cristian Danescu-Niculescu-Mizil, Dan Jurafsky. How to Ask for a Favor: A Case Study on the Success of Altruistic Requests, Proceedings of ICWSM, 2014.
_______________________________________________________________________________________________

## Notebook Title: EDA & Model Baseline
#### Purpose: Load the 'Random Acts of Pizza' train and test data. Conduct an exploratory data analysis to gain an understanding of the data. Create a baseline Logisitic Regression model using non-text (numeric) fields only. 

## I. Load Data and Modules, Process Data

### A. Load Data and Modules

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib.colors import ListedColormap
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from subprocess import check_output
#from wordcloud import WordCloud, STOPWORDS

#ML
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GMM
from sklearn.mixture import GaussianMixture

from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

from sklearn.pipeline import make_pipeline

%matplotlib inline
plt.style.use('bmh')

In [2]:
#1. Train Data
with open('../data/train.json') as fin:
    trainjson = json.load(fin)
train = pd.io.json.json_normalize(trainjson)
#2. Test Data
with open('../data/test.json') as fin:
    testjson = json.load(fin)
test = pd.io.json.json_normalize(testjson)

print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

Train Shape: (4040, 32)
Test Shape: (1631, 17)


### B1. Find any missing values

In [3]:
train.isnull().sum()

giver_username_if_known                                    0
number_of_downvotes_of_request_at_retrieval                0
number_of_upvotes_of_request_at_retrieval                  0
post_was_edited                                            0
request_id                                                 0
request_number_of_comments_at_retrieval                    0
request_text                                               0
request_text_edit_aware                                    0
request_title                                              0
requester_account_age_in_days_at_request                   0
requester_account_age_in_days_at_retrieval                 0
requester_days_since_first_post_on_raop_at_request         0
requester_days_since_first_post_on_raop_at_retrieval       0
requester_number_of_comments_at_request                    0
requester_number_of_comments_at_retrieval                  0
requester_number_of_comments_in_raop_at_request            0
requester_number_of_comm

No missing data except in column "requester_user_flair." We see in the next section that this isn't a column in the test data, so we may just elect to not use it to train the model.

### B2. Identify common columns between test and train

In [4]:
print("Common columns in train and test:")
print(train.columns[train.columns.isin(test.columns)])
print("----")
print("Columns in train but NOT test:")
print(train.columns[~train.columns.isin(test.columns)])

Common columns in train and test:
Index(['giver_username_if_known', 'request_id', 'request_text_edit_aware',
       'request_title', 'requester_account_age_in_days_at_request',
       'requester_days_since_first_post_on_raop_at_request',
       'requester_number_of_comments_at_request',
       'requester_number_of_comments_in_raop_at_request',
       'requester_number_of_posts_at_request',
       'requester_number_of_posts_on_raop_at_request',
       'requester_number_of_subreddits_at_request',
       'requester_subreddits_at_request',
       'requester_upvotes_minus_downvotes_at_request',
       'requester_upvotes_plus_downvotes_at_request', 'requester_username',
       'unix_timestamp_of_request', 'unix_timestamp_of_request_utc'],
      dtype='object')
----
Columns in train but NOT test:
Index(['number_of_downvotes_of_request_at_retrieval',
       'number_of_upvotes_of_request_at_retrieval', 'post_was_edited',
       'request_number_of_comments_at_retrieval', 'request_text',
       '

As can be seen above, there is a series of columns in the training data only. These columns reflect data about the post (e.g., the #of upvotes) at the time this Reddit data was retrieved. We use certain supervised and unsupversied techniques to derive value from this data (even though that information is not provided on the data set we will be predicting).

### C. Create training data, labels, and special 'in training only' data

In [5]:
train_labels_master = train[['requester_received_pizza']]
train_data_master = train[test.columns & train.columns]
train_only_data_master = train[train.columns[~train.columns.isin(test.columns)]].drop(['requester_received_pizza'], axis = 1)

In [6]:
print(train.shape, train_data_master.shape)

(4040, 32) (4040, 17)


### D. Set column types and profile

In [7]:
train_data_master = train_data_master.assign(
    unix_timestamp_of_request = pd.to_datetime(
        train_data_master.unix_timestamp_of_request, unit = "s"),
    unix_timestamp_of_request_utc = pd.to_datetime(
        train_data_master.unix_timestamp_of_request_utc, unit = "s"))

In [8]:
train_data_master.describe()
train_data_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4040 entries, 0 to 4039
Data columns (total 17 columns):
giver_username_if_known                               4040 non-null object
request_id                                            4040 non-null object
request_text_edit_aware                               4040 non-null object
request_title                                         4040 non-null object
requester_account_age_in_days_at_request              4040 non-null float64
requester_days_since_first_post_on_raop_at_request    4040 non-null float64
requester_number_of_comments_at_request               4040 non-null int64
requester_number_of_comments_in_raop_at_request       4040 non-null int64
requester_number_of_posts_at_request                  4040 non-null int64
requester_number_of_posts_on_raop_at_request          4040 non-null int64
requester_number_of_subreddits_at_request             4040 non-null int64
requester_subreddits_at_request                       4040 non-null obj

## V. Analysis of Post Titles

### Isolate Post Subject Line

In [21]:
# Isolate the subject line column for the training and dev dataframes

x_train_titles, x_test_titles, y_train, y_test = train_test_split(
    train_data_master['request_title'], 
    train_labels_master.values.ravel(), test_size=0.29, random_state=0)

import re
from sklearn.grid_search import GridSearchCV
# print(x_train_text)

### Run text cleaning preprocessing on the training dataset to remove a lot of the junk.

# Define custom text preprocessor
def title_cleaner(s):
    # Establish a compiled regex that finds words shorter than 3 characters
    shortword = re.compile(r'\W*\b\w{1,3}\b')
    
    # Convert all text to lowercase
    text = s.lower()
    
    # The word 'request' appears a large number of times, so we'll remove it.
    text = re.sub('request','',text)
    
    # Remove newlines and punctuation marks
    text = re.sub(r'\n', ' ', text)
    text = re.sub('[,?]',' ',text)
    text = re.sub('\. ',' ',text)
    text = re.sub(' \.',' ',text)
    text = re.sub('\.{2,}',' ',text)
    text = re.sub(r'/',' ',text)
    text = re.sub('-','',text)
    text = re.sub('"','',text)
    text = re.sub('[<>()]',' ',text)

    # Convert sequences of numbers to zero
    text = re.sub('\d+', '0', text)
    
    # Remove short words
    text = shortword.sub('', text)
    
    # Remove extra whitespace
    text = re.sub(' +',' ',text)

    return text

# Set up count vectorizer to use custom preprocessor. We'll try analyzing both with individual words and with bigrams.

vectimus_maximus = CountVectorizer(analyzer='word',ngram_range=(1,1)) 
vector_supreme = CountVectorizer(analyzer='word',ngram_range=(2,2)) 

x_train_vect = vectimus_maximus.fit_transform(x_train_titles)
x_test_vect = vectimus_maximus.transform(x_test_titles)
# print(vectimus_maximus.vocabulary_)
                                         
bigram_train_vect = vector_supreme.fit_transform(x_train_titles)
bigram_test_vect = vector_supreme.transform(x_test_titles)
                                         
# # Fit a Bernoulli Naive Bayes model using the vectorized text and use GridSearch to optimize params
# model_TextNB = BernoulliNB()
# alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 20.0, 50.0, 100.0]}
# BernNB_clf = GridSearchCV(model_TextNB,param_grid=alphas)
# BernNB_clf.fit(x_train_vect,y_train)
# print('Optimized score for BernoulliNB (alpha=',BernNB_clf.best_params_['alpha'],'): ',
#       BernNB_clf.best_score_,'\n',sep='')
# alpha_optimal = BernNB_clf.best_params_['alpha']

# Predict and check accuracy
model_TextNB = BernoulliNB()
model_TextNB.fit(x_train_vect,y_train)
predict_NB = model_TextNB.predict(x_test_vect)
test_accNB = metrics.accuracy_score(y_test, predict_NB)
print(test_accNB)

# score_NB = model_TextNB.score(x_test_vect, y_test)
# print(score_NB)

model_TextNB2 = BernoulliNB()
model_TextNB2.fit(bigram_train_vect,y_train)
predict_NB2 = model_TextNB2.predict(bigram_test_vect)
test_accNB2 = metrics.accuracy_score(y_test, predict_NB2)
print(test_accNB2)

0.7525597269624573
0.7602389078498294
